# Pulling Weather Data from the API and Merging with Generation and Capacity Data

In [229]:
# Environments
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pytz
from datetime import datetime
from pvlive_api import PVLive
import time
from datetime import datetime, timedelta, timezone
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [230]:
# Initiating PVLive API as per GIT repo instructions: https://github.com/SheffieldSolar/PV_Live-API
pvl = PVLive(
    retries=3, # Optionally set the number of retries when intermittent issues are encountered
    proxies=None, # Optionally pass a dict of proxies to use when making requests
    ssl_verify=True, # Optionally disable SSL certificate verification (not advised!)
    domain_url="api.pvlive.uk", # Optionally switch between the prod and FOF APIs
)

In [231]:
def load_mwp(region="gsp", include_history=True):
    """
    Load and return the MWp deployment dataframe as mwp_df via pvl.deployment.
    """
    mwp_df = pvl.deployment(region=region, include_history=include_history)
    return mwp_df

def load_gsp(gsp_path="../data/gsp_info.csv"):
    """
    Load and return the GSP info dataframe as gsp_df from CSV,
    filtered to only GSPs known to PVLive (pvl.gsp_ids).
    """
    pvl = PVLive()
    valid_ids = set(pvl.gsp_ids)

    gsp_df = pd.read_csv(gsp_path)

    if 'gsp_id' in gsp_df.columns:
        # coerce non-numeric to NaN, drop those rows, cast to int, then filter by PVLive ids
        gsp_df['gsp_id_num'] = pd.to_numeric(gsp_df['gsp_id'], errors='coerce')
        gsp_df = gsp_df[gsp_df['gsp_id_num'].notna()].copy()
        gsp_df['gsp_id_num'] = gsp_df['gsp_id_num'].astype(int)
        gsp_df = gsp_df[gsp_df['gsp_id_num'].isin(valid_ids)].drop(columns=['gsp_id_num']).reset_index(drop=True)

    return gsp_df

def merge_gsp_location(mwp_df, gsp_df, gsp_col_mwp='GSPs', gsp_col_gsp='gsp_name'):
    """
    Return a copy of mwp_df with columns gsp_lat, gsp_lon, region_name merged from gsp_df.
    Matching is done case-insensitive and with whitespace stripped.
    Remove the 'unkown' rows from the mwp_df - presumably misspelling of unknown.
    Drop any rows with missing values.
    
    """
    # Make copies to avoid mutating inputs
    mwp = mwp_df.copy()
    gsp = gsp_df.copy()

    # Normalize join keys by aligning to string, stripping and putting in upper case
    mwp['_gsp_key'] = mwp[gsp_col_mwp].astype(str).str.strip().str.upper()
    gsp['_gsp_key'] = gsp[gsp_col_gsp].astype(str).str.strip().str.upper()

    # Select only the columns we want to bring across (plus join key)
    to_merge = gsp[['_gsp_key', 'gsp_id', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']].drop_duplicates('_gsp_key')    
    merged = mwp.merge(to_merge, on='_gsp_key', how='left') # Left merge so all mwp rows are kept
    merged = merged.dropna(how='any')   # drop all rows where ther are NaN values - return only the 299 intersection GSPs
    merged = merged[merged[gsp_col_mwp] != 'unkown']  # return the df where not equal to unkown
    merged = merged.drop(columns=['_gsp_key'])  # Drop linking key

    return merged
    
def gsp_locations(merged_df, gsp_col='GSPs'):
    """
    Return a DataFrame with one row per unique GSP containing
    gsp_col, gsp_lat, gsp_lon, region_name, pes_id, and a combined
    'GSP_region' column formatted "GSPs | region_name".
    """
    gsp_locations_list = (
        merged_df
        .drop_duplicates(subset=[gsp_col])[[gsp_col, 'gsp_id', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']]
        .reset_index(drop=True)
    )
    gsp_locations_list['GSP_region'] = gsp_locations_list[gsp_col].astype(str) + ' | ' + gsp_locations_list['region_name'].astype(str)
    return gsp_locations_list

def wide_cumul_capacity(merged_df, time_col='install_month', gsp_col='GSPs', value_col='cumul_capacity_mwp'):
    """
    Return a DataFrame with:
    - one column for the time (time_col) monthly intervals
    - one column per GSP (column name = GSP identifier)
    - cells = value_col (cumulative capacity MWP)
    """
    import pandas as pd
    df = merged_df.copy()
    try:
        df[time_col] = pd.to_datetime(df[time_col])
    except Exception:
        pass
    wide = df.pivot_table(index=time_col, columns=gsp_col, values=value_col, aggfunc='first')
    wide = wide.reset_index()  # make time a regular column
    return wide

def download_generation_for_single_gsp(start, end, gsp_id, gsp_locations_list, include_national=False, extra_fields=""):
    """
    Return a DataFrame with:
    - generation data for selected period for one GSP
    - one column for the time (time_col) HH intervals
    - other columns for GSP identifiers
    """
    valid_ids = gsp_locations_list['gsp_id'].dropna().astype(int).unique()  # Get the valid gsp_ids from gsp_locations_list
    if gsp_id not in valid_ids:  # Check if the provided gsp_id is valid
        return f"Please select a GSP ID which appears in the GSP capacity list."

    # Fetch data for the specific GSP ID using between function from PVLive API Class
    generation_df = pvl.between(
        start=start,
        end=end,
        entity_type="gsp",
        entity_id=int(gsp_id),
        dataframe=True,
        extra_fields=extra_fields
    )

    # Interpolate up to 12 consecutive NaN values in the generation output - 6 in either direction  
    if generation_df is not None and not generation_df.empty:
        generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
        generation_df = generation_df.sort_values(['gsp_id', 'datetime_gmt']).set_index('datetime_gmt')
        generation_df['generation_mw'] = (
            generation_df.groupby('gsp_id')['generation_mw']
            .apply(lambda s: s.interpolate(method='time', limit=12, limit_direction='both'))
            .reset_index(level=0, drop=True)
        )
        generation_df = generation_df.reset_index()
        
    # Merge additional columns from gsp_locations_list
    gsp_info = gsp_locations_list[gsp_locations_list['gsp_id'] == gsp_id]
    if not gsp_info.empty:
        # Merge on gsp_id to include other columns like gsp_lat, gsp_lon, etc.
        generation_df = generation_df.merge(gsp_info, on='gsp_id', how='left')
        
    return generation_df

def get_capacity_data_single_gsp(gsp_id, merged_df):
    """
    Return a DataFrame containing capacity data for the specified GSP ID,
    along with month and year columns based on the install_month
    
    """
    # Filter the DataFrame for the specified GSP ID and add time-series columns for mathing with generation df
    capacity_data = merged_df[merged_df['gsp_id'] == gsp_id].copy()
    capacity_data = capacity_data[['install_month', 'cumul_capacity_mwp', 'GSPs', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']]   # Keep relevant columns
    capacity_data['install_month'] = pd.to_datetime(capacity_data['install_month'])    # Convert install_month to datetime
    capacity_data['month'] = capacity_data['install_month'].dt.month     # Create 'month' and 'year' columns
    capacity_data['year'] = capacity_data['install_month'].dt.year
    capacity_data['day'] = capacity_data['install_month'].dt.day
    capacity_data['hour'] = capacity_data['install_month'].dt.hour
    return capacity_data.reset_index(drop=True)

def add_capacity_to_generation(generation_df, capacity_data):
    """
    Returns dataframe which merges cumulative capacity and generation data for a single GSP
    
    """
    # Ensure datetime_gmt is in datetime format and extract month/year/day/hour
    generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
    generation_df['month'] = generation_df['datetime_gmt'].dt.month
    generation_df['year'] = generation_df['datetime_gmt'].dt.year
    generation_df['day'] = generation_df['datetime_gmt'].dt.day
    generation_df['hour'] = generation_df['datetime_gmt'].dt.hour
    
    # Merge capacity data based on the month and year values
    merged_df = generation_df.merge(
        capacity_data[['month', 'year', 'cumul_capacity_mwp']], 
        on=['month', 'year'], 
        how='left'
    )
    
    merged_df.rename(columns={'cumul_capacity_mwp': 'capacity_mwp'}, inplace=True)  # rename capacity column
    merged_df['capacity_mwp'] = merged_df['capacity_mwp'].ffill() # replace last NaN values with final entry filling delayed capacity information
    
    return merged_df

# Set GSP and timeframes to call functions
start = datetime(2025, 1, 1, 0, 0, tzinfo=pytz.UTC)
end = datetime(2025, 12, 31, 23, 30, tzinfo=pytz.UTC)
gsp_id = 12  # select GSP ID to extract data for


# Putting the Functions into Variables

In [232]:
# Call functions and create variables with dataframes for use in the app
mwp_df = load_mwp() # loading the capacity df
gsp_df = load_gsp() # loading the gsp locations df
merged_df = merge_gsp_location(mwp_df, gsp_df) # merge capacity growth and locations
gsp_locations_list = gsp_locations(merged_df) # merge capacity and locations without capacity growth over time
generation_df = download_generation_for_single_gsp(start, end, gsp_id, gsp_locations_list) # generation df for selected gsp
capacity_growth_all_gsps = wide_cumul_capacity(merged_df) # wide capacity growth df for all time and all gsps
capacity_data_single_gsp = get_capacity_data_single_gsp(gsp_id, merged_df) # add month and year to capacity single gsp
generation_and_capacity_single_gsp = add_capacity_to_generation(generation_df, capacity_data_single_gsp) # merged capacity and generation same time-series single gsp

generation_and_capacity_single_gsp # extract the gsp generation and capacity data

,datetime_gmt,gsp_id,generation_mw,GSPs,gsp_lat,gsp_lon,region_name,pes_id,GSP_region,month,year,day,hour,capacity_mwp
0,2025-01-01 00:00:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
1,2025-01-01 00:30:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
2,2025-01-01 01:00:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
3,2025-01-01 01:30:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
4,2025-01-01 02:00:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,2,7.659577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17507,2025-12-31 21:30:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,21,8.206667
17508,2025-12-31 22:00:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,22,8.206667
17509,2025-12-31 22:30:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,22,8.206667
17510,2025-12-31 23:00:00+00:00,12,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,23,8.206667


In [233]:
# Any NaNs?
print("Any NaNs:", generation_and_capacity_single_gsp.isna().any().any())

# Count NaNs per column
print("\nNaN count per column:")
print(generation_and_capacity_single_gsp.isna().sum())

Any NaNs: False

NaN count per column:
datetime_gmt     0
gsp_id           0
generation_mw    0
GSPs             0
gsp_lat          0
gsp_lon          0
region_name      0
pes_id           0
GSP_region       0
month            0
year             0
day              0
hour             0
capacity_mwp     0
dtype: int64


In [234]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-01-01",
	"end_date": "2026-01-19",
	"hourly": ["shortwave_radiation", "direct_radiation", "diffuse_radiation", "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation", "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant", "direct_normal_irradiance_instant", "global_tilted_irradiance_instant", "terrestrial_radiation_instant"],
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()
hourly_direct_radiation = hourly.Variables(1).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(2).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(3).ValuesAsNumpy()
hourly_global_tilted_irradiance = hourly.Variables(4).ValuesAsNumpy()
hourly_terrestrial_radiation = hourly.Variables(5).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(6).ValuesAsNumpy()
hourly_direct_radiation_instant = hourly.Variables(7).ValuesAsNumpy()
hourly_diffuse_radiation_instant = hourly.Variables(8).ValuesAsNumpy()
hourly_direct_normal_irradiance_instant = hourly.Variables(9).ValuesAsNumpy()
hourly_global_tilted_irradiance_instant = hourly.Variables(10).ValuesAsNumpy()
hourly_terrestrial_radiation_instant = hourly.Variables(11).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
hourly_data["direct_radiation"] = hourly_direct_radiation
hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)


Coordinates: 52.52000045776367°N 13.419998168945312°E
Elevation: 38.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                           date  shortwave_radiation  direct_radiation  \
0    2025-01-01 00:00:00+00:00                  0.0               0.0   
1    2025-01-01 01:00:00+00:00                  0.0               0.0   
2    2025-01-01 02:00:00+00:00                  0.0               0.0   
3    2025-01-01 03:00:00+00:00                  0.0               0.0   
4    2025-01-01 04:00:00+00:00                  0.0               0.0   
...                        ...                  ...               ...   
9211 2026-01-19 19:00:00+00:00                  0.0               0.0   
9212 2026-01-19 20:00:00+00:00                  0.0               0.0   
9213 2026-01-19 21:00:00+00:00                  0.0               0.0   
9214 2026-01-19 22:00:00+00:00                  0.0               0.0   
9215 2026-01-19 23:00:00+00:00                  0.0               0.0   



# 1.0 Turning the historical weather API into a function

In [235]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

def fetch_openmeteo_hourly_df(latitude,
                              longitude,
                              start_date,
                              end_date,
                              cache_path=".cache",
                              cache_expire=3600,
                              retries=5,
                              backoff_factor=0.2):
    """
    Fetch hourly Open-Meteo historical forecast data using openmeteo_requests.Client
    and return a pandas DataFrame of hourly variables (indexed by UTC datetime).
    """
    cache_session = requests_cache.CachedSession(cache_path, expire_after=cache_expire)
    retry_session = retry(cache_session, retries=retries, backoff_factor=backoff_factor)
    client = openmeteo_requests.Client(session=retry_session)

    # Workaround for openmeteo_requests.Client.__del__ bug
    if not hasattr(client, "_close_session"):
        client._close_session = False

    hourly_vars = [
        "shortwave_radiation", "direct_radiation", "diffuse_radiation",
        "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation",
        "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant",
        "direct_normal_irradiance_instant", "global_tilted_irradiance_instant",
        "terrestrial_radiation_instant",
    ]

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": hourly_vars,
    }

    responses = client.weather_api(url, params=params)
    if not responses:
        return pd.DataFrame()

    response = responses[0]
    hourly = response.Hourly()
    hourly_arrays = [hourly.Variables(i).ValuesAsNumpy() for i in range(len(hourly_vars))]

    start_ts = pd.to_datetime(hourly.Time(), unit="s", utc=True)
    end_ts = pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True)
    interval_seconds = int(hourly.Interval())
    dates = pd.date_range(
        start=start_ts,
        end=end_ts,
        freq=pd.Timedelta(seconds=interval_seconds),
        inclusive="left"
    )

    data = {"date": dates}
    for name, arr in zip(hourly_vars, hourly_arrays):
        data[name] = arr[: len(dates)]

    df = pd.DataFrame(data).set_index("date")
    return df


In [236]:
# Call the function (assuming fetch_openmeteo_hourly_df is defined in the same module)
df = fetch_openmeteo_hourly_df(
    latitude=52.52,
    longitude=13.41,
    start_date="2025-01-01",
    end_date="2026-01-19"
)

df.head()

,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
date,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 1.1 Weather API function calling lat and long and dates from generation df

In [237]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

def fetch_weather_for_generation_df(gen_df,
                                    cache_path=".cache",
                                    cache_expire=3600,
                                    retries=5,
                                    backoff_factor=0.2):
    """
    Extract lat/lon and date range from generation_and_capacity_single_gsp-like
    DataFrame (expects 'gsp_lat', 'gsp_lon', 'datetime_gmt'), call Open-Meteo via
    openmeteo_requests and return hourly weather DataFrame indexed by UTC datetime.
    """
    # Extract lat/lon from first non-null row
    row = gen_df[['gsp_lat', 'gsp_lon']].dropna().iloc[0]
    latitude, longitude = float(row['gsp_lat']), float(row['gsp_lon'])

    # Derive start/end dates (YYYY-MM-DD)
    start_date = gen_df['datetime_gmt'].min().normalize().strftime('%Y-%m-%d')
    end_date = gen_df['datetime_gmt'].max().normalize().strftime('%Y-%m-%d')

    # Setup client with cache + retry
    cache_session = requests_cache.CachedSession(cache_path, expire_after=cache_expire)
    retry_session = retry(cache_session, retries=retries, backoff_factor=backoff_factor)
    client = openmeteo_requests.Client(session=retry_session)
    if not hasattr(client, "_close_session"):
        client._close_session = False  # avoid destructor AttributeError

    hourly_vars = [
        "shortwave_radiation", "direct_radiation", "diffuse_radiation",
        "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation",
        "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant",
        "direct_normal_irradiance_instant", "global_tilted_irradiance_instant",
        "terrestrial_radiation_instant",
    ]

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": hourly_vars,
    }

    responses = client.weather_api(url, params=params)
    if not responses:
        return pd.DataFrame()

    response = responses[0]
    hourly = response.Hourly()
    hourly_arrays = [hourly.Variables(i).ValuesAsNumpy() for i in range(len(hourly_vars))]

    start_ts = pd.to_datetime(hourly.Time(), unit="s", utc=True)
    end_ts = pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True)
    interval_seconds = int(hourly.Interval())
    dates = pd.date_range(
        start=start_ts,
        end=end_ts,
        freq=pd.Timedelta(seconds=interval_seconds),
        inclusive="left"
    )

    data = {"date": dates}
    for name, arr in zip(hourly_vars, hourly_arrays):
        data[name] = arr[: len(dates)]

    df = pd.DataFrame(data).set_index("date")
    return df

# Usage:
weather_df = fetch_weather_for_generation_df(generation_and_capacity_single_gsp)
weather_df.head()


,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
date,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [238]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

def fetch_weather_for_generation_df(gen_df,
                                    cache_path=".cache",
                                    cache_expire=3600,
                                    retries=5,
                                    backoff_factor=0.2):
    """
    Extract lat/lon and date range from generation_and_capacity_single_gsp-like
    DataFrame (expects 'gsp_lat', 'gsp_lon', 'datetime_gmt'), call Open-Meteo via
    openmeteo_requests and return hourly weather DataFrame indexed by UTC datetime
    with added columns: year, month, day, hour.
    """
    # Extract lat/lon from first non-null row
    row = gen_df[['gsp_lat', 'gsp_lon']].dropna().iloc[0]
    latitude, longitude = float(row['gsp_lat']), float(row['gsp_lon'])

    # Derive start/end dates (YYYY-MM-DD)
    start_date = gen_df['datetime_gmt'].min().normalize().strftime('%Y-%m-%d')
    end_date = gen_df['datetime_gmt'].max().normalize().strftime('%Y-%m-%d')

    # Setup client with cache + retry
    cache_session = requests_cache.CachedSession(cache_path, expire_after=cache_expire)
    retry_session = retry(cache_session, retries=retries, backoff_factor=backoff_factor)
    client = openmeteo_requests.Client(session=retry_session)
    if not hasattr(client, "_close_session"):
        client._close_session = False  # avoid destructor AttributeError

    hourly_vars = [
        "shortwave_radiation", "direct_radiation", "diffuse_radiation",
        "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation",
        "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant",
        "direct_normal_irradiance_instant", "global_tilted_irradiance_instant",
        "terrestrial_radiation_instant",
    ]

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": hourly_vars,
    }

    responses = client.weather_api(url, params=params)
    if not responses:
        return pd.DataFrame()

    response = responses[0]
    hourly = response.Hourly()
    hourly_arrays = [hourly.Variables(i).ValuesAsNumpy() for i in range(len(hourly_vars))]

    start_ts = pd.to_datetime(hourly.Time(), unit="s", utc=True)
    end_ts = pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True)
    interval_seconds = int(hourly.Interval())
    dates = pd.date_range(
        start=start_ts,
        end=end_ts,
        freq=pd.Timedelta(seconds=interval_seconds),
        inclusive="left"
    )

    data = {"date": dates}
    for name, arr in zip(hourly_vars, hourly_arrays):
        data[name] = arr[: len(dates)]

    df = pd.DataFrame(data).set_index("date")

    # Add year, month, day, hour columns from the index
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hour'] = df.index.hour

    return df

# Usage:
weather_df = fetch_weather_for_generation_df(generation_and_capacity_single_gsp)
weather_df.head()


,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant,year,month,day,hour
date,,,,,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,0
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,1
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,2
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,3
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,4


In [239]:
def merge_generation_with_weather(generation_df, weather_df):
    """
    Merge generation_and_capacity_single_gsp dataframe with weather_df.
    Only uses generation_mw and capacity_mwp from generation_df.
    - generation_df: must contain ['year','month','day','hour','generation_mw','capacity_mwp']
      (two half-hour rows per hour; this function averages per hour).
    - weather_df: indexed by UTC datetime and must contain columns ['year','month','day','hour']
    Returns: DataFrame with weather_df index and merged averaged columns:
             generation_mw, capacity_mwp (aligned to weather hourly rows).
    """
    # Select only required columns
    cols = ['year', 'month', 'day', 'hour', 'generation_mw', 'capacity_mwp']
    gen = generation_and_capacity_single_gsp[cols].copy()

    # Aggregate to hourly by mean
    grouped = gen.groupby(['year', 'month', 'day', 'hour'], as_index=False)[['generation_mw', 'capacity_mwp']].mean()

    # Merge with weather_df on year,month,day,hour
    weather = weather_df.copy().reset_index()  # bring datetime index to column named 'date'
    merged = weather.merge(grouped, on=['year', 'month', 'day', 'hour'], how='left')

    # restore datetime index
    merged = merged.set_index('date')

    return merged
gen_weather_merged_df = merge_generation_with_weather(generation_and_capacity_single_gsp, weather_df)
gen_weather_merged_df

,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant,year,month,day,hour,generation_mw,capacity_mwp
date,,,,,,,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,0,0.0,7.659577
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,1,0.0,7.659577
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,2,0.0,7.659577
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,3,0.0,7.659577
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,4,0.0,7.659577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 19:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,19,0.0,8.206667
2025-12-31 20:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,20,0.0,8.206667
2025-12-31 21:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,21,0.0,8.206667


In [240]:
gen_weather_merged_df = merge_generation_with_weather(generation_and_capacity_single_gsp, weather_df)
gen_weather_merged_df

,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant,year,month,day,hour,generation_mw,capacity_mwp
date,,,,,,,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,0,0.0,7.659577
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,1,0.0,7.659577
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,2,0.0,7.659577
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,3,0.0,7.659577
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,4,0.0,7.659577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 19:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,19,0.0,8.206667
2025-12-31 20:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,20,0.0,8.206667
2025-12-31 21:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,21,0.0,8.206667


In [241]:
# Summary of NaNs in gen_weather_merged_df
print("Any NaNs anywhere:", gen_weather_merged_df.isna().any().any())
print("\nNaN count per column:")
print(gen_weather_merged_df.isna().sum())

# Number of rows with any NaN
n_rows_with_nan = gen_weather_merged_df.isna().any(axis=1).sum()
print(f"\nRows with any NaN: {n_rows_with_nan} of {len(gen_weather_merged_df)}")

# Show all rows that contain any NaN (may be large)
nan_rows = gen_weather_merged_df[gen_weather_merged_df.isna().any(axis=1)]
print("\nRows with NaNs (showing up to 200):")
print(nan_rows.head(200))

# If you only care about generation_mw and capacity_mwp:
print("\nNaNs in generation_mw or capacity_mwp:")
print(gen_weather_merged_df[['generation_mw','capacity_mwp']].isna().sum())
print(gen_weather_merged_df[gen_weather_merged_df[['generation_mw','capacity_mwp']].isna().any(axis=1)].head(200))


Any NaNs anywhere: True

NaN count per column:
shortwave_radiation                 0
direct_radiation                    0
diffuse_radiation                   0
direct_normal_irradiance            0
global_tilted_irradiance            0
terrestrial_radiation               0
shortwave_radiation_instant         0
direct_radiation_instant            0
diffuse_radiation_instant           0
direct_normal_irradiance_instant    0
global_tilted_irradiance_instant    0
terrestrial_radiation_instant       0
year                                0
month                               0
day                                 0
hour                                0
generation_mw                       3
capacity_mwp                        3
dtype: int64

Rows with any NaN: 3 of 8760

Rows with NaNs (showing up to 200):
                           shortwave_radiation  direct_radiation  \
date                                                               
2025-11-03 14:00:00+00:00               100.75       

In [242]:
import pandas as pd

# ensure datetime column is datetime dtype
df = generation_and_capacity_single_gsp.copy()
df['datetime_gmt'] = pd.to_datetime(df['datetime_gmt'])

# Option 1 — counts per year-month (Period)
counts = df.groupby(df['datetime_gmt'].dt.to_period('M')).size().sort_index()
print(counts)

# Option 2 — counts per YYYY-MM string
counts2 = df['datetime_gmt'].dt.strftime('%Y-%m').value_counts().sort_index()
print(counts2)


datetime_gmt
2025-01    1488
2025-02    1344
2025-03    1488
2025-04    1440
2025-05    1488
2025-06    1440
2025-07    1488
2025-08    1488
2025-09    1440
2025-10    1488
2025-11    1432
2025-12    1488
Freq: M, dtype: int64
datetime_gmt
2025-01    1488
2025-02    1344
2025-03    1488
2025-04    1440
2025-05    1488
2025-06    1440
2025-07    1488
2025-08    1488
2025-09    1440
2025-10    1488
2025-11    1432
2025-12    1488
Name: count, dtype: int64


C:\Users\SamF\AppData\Local\Temp\ipykernel_31144\690966510.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  counts = df.groupby(df['datetime_gmt'].dt.to_period('M')).size().sort_index()


In [243]:
import pandas as pd

df = generation_and_capacity_single_gsp.copy()
df['datetime_gmt'] = pd.to_datetime(df['datetime_gmt']).dt.tz_convert('UTC')

start = pd.to_datetime('2025-11-01').tz_localize('UTC')
end   = pd.to_datetime('2025-11-30 23:30:00').tz_localize('UTC')
full_idx = pd.date_range(start, end, freq='30min', tz='UTC')

present = pd.DatetimeIndex(df.loc[(df['datetime_gmt'] >= start) & (df['datetime_gmt'] <= end), 'datetime_gmt'])
missing = full_idx.difference(present)

print("Missing count:", len(missing))
for ts in missing:
    print(ts.isoformat())


Missing count: 8
2025-11-03T13:30:00+00:00
2025-11-03T14:00:00+00:00
2025-11-03T14:30:00+00:00
2025-11-03T15:00:00+00:00
2025-11-03T15:30:00+00:00
2025-11-03T16:00:00+00:00
2025-11-03T16:30:00+00:00
2025-11-03T17:00:00+00:00


# Solving missing date values from merged generation and capacity df (03.11.2025)

In [244]:
def add_capacity_to_generation(generation_df, capacity_data, tz='UTC', gen_fill_method='interpolate'):
    """
    Merge cumulative capacity into generation_df, then reindex to a full 30-minute UTC grid
    and interpolate missing generation_mw values. Returns DataFrame indexed by datetime_gmt (tz-aware).
    
    Parameters:
    - generation_df: DataFrame with 'datetime_gmt' and 'generation_mw'
    - capacity_data: DataFrame with ['month','year','cumul_capacity_mwp']
    - tz: timezone for indexing (default 'UTC')
    - gen_fill_method: 'interpolate' (default) or 'zero' to fill generation gaps
    """
    # ensure datetime and timezone
    generation_df = generation_df.copy()
    generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
    if generation_df['datetime_gmt'].dt.tz is None:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_localize(tz)
    else:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_convert(tz)

    # add time components
    generation_df['month'] = generation_df['datetime_gmt'].dt.month
    generation_df['year'] = generation_df['datetime_gmt'].dt.year
    generation_df['day'] = generation_df['datetime_gmt'].dt.day
    generation_df['hour'] = generation_df['datetime_gmt'].dt.hour

    # merge capacity (by month/year) and forward-fill capacity
    merged_df = generation_df.merge(
        capacity_data[['month', 'year', 'cumul_capacity_mwp']],
        on=['month', 'year'],
        how='left'
    ).rename(columns={'cumul_capacity_mwp': 'capacity_mwp'})
    merged_df['capacity_mwp'] = merged_df['capacity_mwp'].ffill()

    # set datetime index and select relevant columns
    merged_df = merged_df.set_index('datetime_gmt').sort_index()
    cols_keep = [c for c in merged_df.columns]  # keep existing columns (including generation_mw and capacity_mwp)

    # build full 30-minute index from min to max
    full_idx = pd.date_range(merged_df.index.min(), merged_df.index.max(), freq='30min', tz=tz)

    # reindex to full grid
    merged_df = merged_df.reindex(full_idx)

    # ensure capacity is ffilled across new rows
    if 'capacity_mwp' in merged_df.columns:
        merged_df['capacity_mwp'] = merged_df['capacity_mwp'].ffill().bfill()

    # fill generation_mw per chosen method
    if 'generation_mw' in merged_df.columns:
        if gen_fill_method == 'interpolate':
            merged_df['generation_mw'] = merged_df['generation_mw'].interpolate(limit_direction='both')
        else:
            merged_df['generation_mw'] = merged_df['generation_mw'].fillna(0)

    # recompute time component columns from index for any newly created rows
    merged_df['month'] = merged_df.index.month
    merged_df['year']  = merged_df.index.year
    merged_df['day']   = merged_df.index.day
    merged_df['hour']  = merged_df.index.hour

    # reset index name to datetime_gmt
    merged_df.index.name = 'datetime_gmt'

    return merged_df.reset_index()


In [245]:
# Assuming generation_df is obtained from download_generation_for_single_gsp
# and capacity_data is obtained from get_capacity_data_single_gsp

# Example of obtaining capacity data for a specific GSP ID
capacity_data = get_capacity_data_single_gsp(gsp_id, merged_df)

# Now merge them using the modified function
merged_output = add_capacity_to_generation(
    generation_df, 
    capacity_data, 
)

# Check the first few rows of the merged output
merged_output.head()


,datetime_gmt,gsp_id,generation_mw,GSPs,gsp_lat,gsp_lon,region_name,pes_id,GSP_region,month,year,day,hour,capacity_mwp
0,2025-01-01 00:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
1,2025-01-01 00:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
2,2025-01-01 01:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
3,2025-01-01 01:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
4,2025-01-01 02:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,2,7.659577


In [246]:
# Check for NaNs in merged_output
print("Any NaNs anywhere in merged_output:", merged_output.isna().any().any())

# Count NaNs per column
print("\nNaN count per column in merged_output:")
print(merged_output.isna().sum())

# Count of rows with any NaN
n_rows_with_nan = merged_output.isna().any(axis=1).sum()
print(f"\nNumber of rows with any NaN: {n_rows_with_nan} out of {len(merged_output)}")

# Show all rows that contain any NaN (up to 20 for a quick check)
nan_rows = merged_output[merged_output.isna().any(axis=1)]
print("\nRows with NaNs (showing up to 20):")
print(nan_rows.head(20))


Any NaNs anywhere in merged_output: True

NaN count per column in merged_output:
datetime_gmt     0
gsp_id           8
generation_mw    0
GSPs             8
gsp_lat          8
gsp_lon          8
region_name      8
pes_id           8
GSP_region       8
month            0
year             0
day              0
hour             0
capacity_mwp     0
dtype: int64

Number of rows with any NaN: 8 out of 17520

Rows with NaNs (showing up to 20):
                   datetime_gmt  gsp_id  generation_mw GSPs  gsp_lat  gsp_lon  \
14715 2025-11-03 13:30:00+00:00     NaN       0.064739  NaN      NaN      NaN   
14716 2025-11-03 14:00:00+00:00     NaN       0.056646  NaN      NaN      NaN   
14717 2025-11-03 14:30:00+00:00     NaN       0.048554  NaN      NaN      NaN   
14718 2025-11-03 15:00:00+00:00     NaN       0.040462  NaN      NaN      NaN   
14719 2025-11-03 15:30:00+00:00     NaN       0.032369  NaN      NaN      NaN   
14720 2025-11-03 16:00:00+00:00     NaN       0.024277  NaN      NaN     

In [247]:
import pandas as pd

df = merged_output.copy()
df['datetime_gmt'] = pd.to_datetime(df['datetime_gmt']).dt.tz_convert('UTC')

start = pd.to_datetime('2025-11-01').tz_localize('UTC')
end   = pd.to_datetime('2025-11-30 23:30:00').tz_localize('UTC')
full_idx = pd.date_range(start, end, freq='30min', tz='UTC')

present = pd.DatetimeIndex(df.loc[(df['datetime_gmt'] >= start) & (df['datetime_gmt'] <= end), 'datetime_gmt'])
missing = full_idx.difference(present)

print("Missing count:", len(missing))
for ts in missing:
    print(ts.isoformat())

Missing count: 0


##### Solving Missing date inputs from generation and capacity df

That's now solved, let's: 

* add it to the working functions list
* complete the merge with weather data again
* check for NaNs again


In [248]:
def add_capacity_to_generation(generation_df, capacity_data, tz='UTC', gen_fill_method='interpolate'):
    """
    Merge cumulative capacity into generation_df, then reindex to a full 30-minute UTC grid
    and interpolate missing generation_mw values. Returns DataFrame indexed by datetime_gmt (tz-aware).
    
    Parameters:
    - generation_df: DataFrame with 'datetime_gmt' and 'generation_mw'
    - capacity_data: DataFrame with ['month', 'year', 'cumul_capacity_mwp']
    - tz: timezone for indexing (default 'UTC')
    - gen_fill_method: 'interpolate' (default) or 'zero' to fill generation gaps
    """
    # Ensure datetime and timezone
    generation_df = generation_df.copy()
    generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
    if generation_df['datetime_gmt'].dt.tz is None:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_localize(tz)
    else:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_convert(tz)

    # Add time components
    generation_df['month'] = generation_df['datetime_gmt'].dt.month
    generation_df['year'] = generation_df['datetime_gmt'].dt.year
    generation_df['day'] = generation_df['datetime_gmt'].dt.day
    generation_df['hour'] = generation_df['datetime_gmt'].dt.hour

    # Merge capacity (by month/year) and forward-fill capacity
    merged_df = generation_df.merge(
        capacity_data[['month', 'year', 'cumul_capacity_mwp']],
        on=['month', 'year'],
        how='left'
    ).rename(columns={'cumul_capacity_mwp': 'capacity_mwp'})
    merged_df['capacity_mwp'] = merged_df['capacity_mwp'].ffill()

    # Set datetime index and select relevant columns, including ffill on specific fields
    merged_df = merged_df.set_index('datetime_gmt').sort_index()

    # Build full 30-minute index from min to max
    full_idx = pd.date_range(merged_df.index.min(), merged_df.index.max(), freq='30min', tz=tz)

    # Reindex to full grid
    merged_df = merged_df.reindex(full_idx)

    # Fill remaining NaNs for generation_mw
    if 'generation_mw' in merged_df.columns:
        if gen_fill_method == 'interpolate':
            merged_df['generation_mw'] = merged_df['generation_mw'].interpolate(limit_direction='both')
        else:
            merged_df['generation_mw'] = merged_df['generation_mw'].fillna(0)
    
    # Fill remaining NaNs for capacity_mw
    if 'capacity_mwp' in merged_df.columns:
        if gen_fill_method == 'interpolate':
            merged_df['capacity_mwp'] = merged_df['capacity_mwp'].interpolate(limit_direction='both')
        else:
            merged_df['capacity_mwp'] = merged_df['capacity_mwp'].fillna(0)
        
    # Forward fill for newly made specific non-numeric columns
    ffill_columns = ['gsp_id', 'GSPs', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id', 'GSP_region']
    for col in ffill_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].ffill()

    # Recompute time component columns from index for any newly created rows
    merged_df['month'] = merged_df.index.month
    merged_df['year'] = merged_df.index.year
    merged_df['day'] = merged_df.index.day
    merged_df['hour'] = merged_df.index.hour

    # Reset index name to 'datetime_gmt'
    merged_df.index.name = 'datetime_gmt'

    return merged_df.reset_index()


In [249]:
test_df = add_capacity_to_generation(generation_df, capacity_data)
test_df

,datetime_gmt,gsp_id,generation_mw,GSPs,gsp_lat,gsp_lon,region_name,pes_id,GSP_region,month,year,day,hour,capacity_mwp
0,2025-01-01 00:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
1,2025-01-01 00:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,0,7.659577
2,2025-01-01 01:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
3,2025-01-01 01:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,1,7.659577
4,2025-01-01 02:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,1,2025,1,2,7.659577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2025-12-31 21:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,21,8.206667
17516,2025-12-31 22:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,22,8.206667
17517,2025-12-31 22:30:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,22,8.206667
17518,2025-12-31 23:00:00+00:00,12.0,0.0,CAMB_01,53.729427,-1.000941,Camblesforth,23.0,CAMB_01 | Camblesforth,12,2025,31,23,8.206667


In [250]:
# Check for NaNs in merged_output
print("Any NaNs anywhere in merged_output:", test_df.isna().any().any())

# Count NaNs per column
print("\nNaN count per column in merged_output:")
print(test_df.isna().sum())

# Count of rows with any NaN
n_rows_with_nan = test_df.isna().any(axis=1).sum()
print(f"\nNumber of rows with any NaN: {n_rows_with_nan} out of {len(test_df)}")

# Show all rows that contain any NaN (up to 20 for a quick check)
nan_rows = test_df[test_df.isna().any(axis=1)]
print("\nRows with NaNs (showing up to 20):")
print(nan_rows.head(20))


Any NaNs anywhere in merged_output: False

NaN count per column in merged_output:
datetime_gmt     0
gsp_id           0
generation_mw    0
GSPs             0
gsp_lat          0
gsp_lon          0
region_name      0
pes_id           0
GSP_region       0
month            0
year             0
day              0
hour             0
capacity_mwp     0
dtype: int64

Number of rows with any NaN: 0 out of 17520

Rows with NaNs (showing up to 20):
Empty DataFrame
Columns: [datetime_gmt, gsp_id, generation_mw, GSPs, gsp_lat, gsp_lon, region_name, pes_id, GSP_region, month, year, day, hour, capacity_mwp]
Index: []


# 2.0 Merge this with all working functions

In [251]:
def load_mwp(region="gsp", include_history=True):
    """
    Load and return the MWp deployment dataframe as mwp_df via pvl.deployment.
    """
    mwp_df = pvl.deployment(region=region, include_history=include_history)
    return mwp_df

def load_gsp(gsp_path="../data/gsp_info.csv"):
    """
    Load and return the GSP info dataframe as gsp_df from CSV,
    filtered to only GSPs known to PVLive (pvl.gsp_ids).
    """
    pvl = PVLive()
    valid_ids = set(pvl.gsp_ids)

    gsp_df = pd.read_csv(gsp_path)

    if 'gsp_id' in gsp_df.columns:
        # coerce non-numeric to NaN, drop those rows, cast to int, then filter by PVLive ids
        gsp_df['gsp_id_num'] = pd.to_numeric(gsp_df['gsp_id'], errors='coerce')
        gsp_df = gsp_df[gsp_df['gsp_id_num'].notna()].copy()
        gsp_df['gsp_id_num'] = gsp_df['gsp_id_num'].astype(int)
        gsp_df = gsp_df[gsp_df['gsp_id_num'].isin(valid_ids)].drop(columns=['gsp_id_num']).reset_index(drop=True)

    return gsp_df

def merge_gsp_location(mwp_df, gsp_df, gsp_col_mwp='GSPs', gsp_col_gsp='gsp_name'):
    """
    Return a copy of mwp_df with columns gsp_lat, gsp_lon, region_name merged from gsp_df.
    Matching is done case-insensitive and with whitespace stripped.
    Remove the 'unkown' rows from the mwp_df - presumably misspelling of unknown.
    Drop any rows with missing values.
    
    """
    # Make copies to avoid mutating inputs
    mwp = mwp_df.copy()
    gsp = gsp_df.copy()

    # Normalize join keys by aligning to string, stripping and putting in upper case
    mwp['_gsp_key'] = mwp[gsp_col_mwp].astype(str).str.strip().str.upper()
    gsp['_gsp_key'] = gsp[gsp_col_gsp].astype(str).str.strip().str.upper()

    # Select only the columns we want to bring across (plus join key)
    to_merge = gsp[['_gsp_key', 'gsp_id', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']].drop_duplicates('_gsp_key')    
    merged = mwp.merge(to_merge, on='_gsp_key', how='left') # Left merge so all mwp rows are kept
    merged = merged.dropna(how='any')   # drop all rows where ther are NaN values - return only the 299 intersection GSPs
    merged = merged[merged[gsp_col_mwp] != 'unkown']  # return the df where not equal to unkown
    merged = merged.drop(columns=['_gsp_key'])  # Drop linking key

    return merged
    
def gsp_locations(merged_df, gsp_col='GSPs'):
    """
    Return a DataFrame with one row per unique GSP containing
    gsp_col, gsp_lat, gsp_lon, region_name, pes_id, and a combined
    'GSP_region' column formatted "GSPs | region_name".
    """
    gsp_locations_list = (
        merged_df
        .drop_duplicates(subset=[gsp_col])[[gsp_col, 'gsp_id', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']]
        .reset_index(drop=True)
    )
    gsp_locations_list['GSP_region'] = gsp_locations_list[gsp_col].astype(str) + ' | ' + gsp_locations_list['region_name'].astype(str)
    return gsp_locations_list

def wide_cumul_capacity(merged_df, time_col='install_month', gsp_col='GSPs', value_col='cumul_capacity_mwp'):
    """
    Return a DataFrame with:
    - one column for the time (time_col) monthly intervals
    - one column per GSP (column name = GSP identifier)
    - cells = value_col (cumulative capacity MWP)
    """
    import pandas as pd
    df = merged_df.copy()
    try:
        df[time_col] = pd.to_datetime(df[time_col])
    except Exception:
        pass
    wide = df.pivot_table(index=time_col, columns=gsp_col, values=value_col, aggfunc='first')
    wide = wide.reset_index()  # make time a regular column
    return wide

def download_generation_for_single_gsp(start, end, gsp_id, gsp_locations_list, include_national=False, extra_fields=""):
    """
    Return a DataFrame with:
    - generation data for selected period for one GSP
    - one column for the time (time_col) HH intervals
    - other columns for GSP identifiers
    """
    valid_ids = gsp_locations_list['gsp_id'].dropna().astype(int).unique()  # Get the valid gsp_ids from gsp_locations_list
    if gsp_id not in valid_ids:  # Check if the provided gsp_id is valid
        return f"Please select a GSP ID which appears in the GSP capacity list."

    # Fetch data for the specific GSP ID using between function from PVLive API Class
    generation_df = pvl.between(
        start=start,
        end=end,
        entity_type="gsp",
        entity_id=int(gsp_id),
        dataframe=True,
        extra_fields=extra_fields
    )

    # Interpolate up to 24 consecutive NaN values in the generation output - 12 in either direction  
    if generation_df is not None and not generation_df.empty:
        generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
        generation_df = generation_df.sort_values(['gsp_id', 'datetime_gmt']).set_index('datetime_gmt')
        generation_df['generation_mw'] = (
            generation_df.groupby('gsp_id')['generation_mw']
            .apply(lambda s: s.interpolate(method='time', limit=12, limit_direction='both'))
            .reset_index(level=0, drop=True)
        )
        generation_df = generation_df.reset_index()
        
    # Merge additional columns from gsp_locations_list
    gsp_info = gsp_locations_list[gsp_locations_list['gsp_id'] == gsp_id]
    if not gsp_info.empty:
        # Merge on gsp_id to include other columns like gsp_lat, gsp_lon, etc.
        generation_df = generation_df.merge(gsp_info, on='gsp_id', how='left')
        
    return generation_df

def get_capacity_data_single_gsp(gsp_id, merged_df):
    """
    Return a DataFrame containing capacity data for the specified GSP ID,
    along with month and year columns based on the install_month
    
    """
    # Filter the DataFrame for the specified GSP ID and add time-series columns for mathing with generation df
    capacity_data = merged_df[merged_df['gsp_id'] == gsp_id].copy()
    capacity_data = capacity_data[['install_month', 'cumul_capacity_mwp', 'GSPs', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id']]   # Keep relevant columns
    capacity_data['install_month'] = pd.to_datetime(capacity_data['install_month'])    # Convert install_month to datetime
    capacity_data['month'] = capacity_data['install_month'].dt.month     # Create 'month' and 'year' columns
    capacity_data['year'] = capacity_data['install_month'].dt.year
    capacity_data['day'] = capacity_data['install_month'].dt.day
    capacity_data['hour'] = capacity_data['install_month'].dt.hour
    return capacity_data.reset_index(drop=True)

def add_capacity_to_generation(generation_df, capacity_data, tz='UTC', gen_fill_method='interpolate'):
    """
    Merge cumulative capacity into generation_df, then reindex to a full 30-minute UTC grid
    and interpolate missing generation_mw and capacity_mwp values. Returns DataFrame indexed by datetime_gmt (tz-aware).
    
    Parameters:
    - generation_df: DataFrame with 'datetime_gmt' and 'generation_mw'
    - capacity_data: DataFrame with ['month', 'year', 'cumul_capacity_mwp']
    - tz: timezone for indexing (default 'UTC')
    - gen_fill_method: 'interpolate' (default) or 'zero' to fill generation gaps
    """
    # Ensure datetime and timezone
    generation_df = generation_df.copy()
    generation_df['datetime_gmt'] = pd.to_datetime(generation_df['datetime_gmt'])
    if generation_df['datetime_gmt'].dt.tz is None:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_localize(tz)
    else:
        generation_df['datetime_gmt'] = generation_df['datetime_gmt'].dt.tz_convert(tz)

    # Add time components
    generation_df['month'] = generation_df['datetime_gmt'].dt.month
    generation_df['year'] = generation_df['datetime_gmt'].dt.year
    generation_df['day'] = generation_df['datetime_gmt'].dt.day
    generation_df['hour'] = generation_df['datetime_gmt'].dt.hour

    # Merge capacity (by month/year) and forward-fill capacity
    merged_df = generation_df.merge(
        capacity_data[['month', 'year', 'cumul_capacity_mwp']],
        on=['month', 'year'],
        how='left'
    ).rename(columns={'cumul_capacity_mwp': 'capacity_mwp'})
    merged_df['capacity_mwp'] = merged_df['capacity_mwp'].ffill()

    # Set datetime index and select relevant columns, including ffill on specific fields
    merged_df = merged_df.set_index('datetime_gmt').sort_index()

    # Build full 30-minute index from min to max
    full_idx = pd.date_range(merged_df.index.min(), merged_df.index.max(), freq='30min', tz=tz)

    # Reindex to full grid
    merged_df = merged_df.reindex(full_idx)

    # Fill remaining NaNs for generation_mw
    if 'generation_mw' in merged_df.columns:
        if gen_fill_method == 'interpolate':
            merged_df['generation_mw'] = merged_df['generation_mw'].interpolate(limit_direction='both')
        else:
            merged_df['generation_mw'] = merged_df['generation_mw'].fillna(0)
    
    # Fill remaining NaNs for capacity_mw
    if 'capacity_mwp' in merged_df.columns:
        if gen_fill_method == 'interpolate':
            merged_df['capacity_mwp'] = merged_df['capacity_mwp'].interpolate(limit_direction='both')
        else:
            merged_df['capacity_mwp'] = merged_df['capacity_mwp'].fillna(0)
        
    # Forward fill for newly made specific non-numeric columns
    ffill_columns = ['gsp_id', 'GSPs', 'gsp_lat', 'gsp_lon', 'region_name', 'pes_id', 'GSP_region']
    for col in ffill_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].ffill()

    # Recompute time component columns from index for any newly created rows
    merged_df['month'] = merged_df.index.month
    merged_df['year'] = merged_df.index.year
    merged_df['day'] = merged_df.index.day
    merged_df['hour'] = merged_df.index.hour

    # Reset index name to 'datetime_gmt'
    merged_df.index.name = 'datetime_gmt'

    return merged_df.reset_index()

# Set GSP and timeframes to call functions
start = datetime(2025, 1, 1, 0, 0, tzinfo=pytz.UTC)
end = datetime(2025, 12, 31, 23, 30, tzinfo=pytz.UTC)
gsp_id = 123  # select GSP ID to extract data for


In [252]:
# Call functions and create variables with dataframes for use in the app
mwp_df = load_mwp() # loading the capacity df
gsp_df = load_gsp() # loading the gsp locations df
merged_df = merge_gsp_location(mwp_df, gsp_df) # merge capacity growth and locations
gsp_locations_list = gsp_locations(merged_df) # merge capacity and locations without capacity growth over time
generation_df = download_generation_for_single_gsp(start, end, gsp_id, gsp_locations_list) # generation df for selected gsp
capacity_growth_all_gsps = wide_cumul_capacity(merged_df) # wide capacity growth df for all time and all gsps
capacity_data_single_gsp = get_capacity_data_single_gsp(gsp_id, merged_df) # add month and year to capacity single gsp
generation_and_capacity_single_gsp = add_capacity_to_generation(generation_df, capacity_data_single_gsp) # merged capacity and generation same time-series single gsp

generation_and_capacity_single_gsp # extract the gsp generation and capacity data

,datetime_gmt,gsp_id,generation_mw,GSPs,gsp_lat,gsp_lon,region_name,pes_id,GSP_region,month,year,day,hour,capacity_mwp
0,2025-01-01 00:00:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,1,2025,1,0,301.845517
1,2025-01-01 00:30:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,1,2025,1,0,301.845517
2,2025-01-01 01:00:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,1,2025,1,1,301.845517
3,2025-01-01 01:30:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,1,2025,1,1,301.845517
4,2025-01-01 02:00:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,1,2025,1,2,301.845517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2025-12-31 21:30:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,12,2025,31,21,408.331217
17516,2025-12-31 22:00:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,12,2025,31,22,408.331217
17517,2025-12-31 22:30:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,12,2025,31,22,408.331217
17518,2025-12-31 23:00:00+00:00,123.0,0.0,CHTE_1,53.22425,-1.38736,Chesterfield,11.0,CHTE_1 | Chesterfield,12,2025,31,23,408.331217


In [253]:
# Check for NaNs in merged_output
print("Any NaNs anywhere in merged_output:", generation_and_capacity_single_gsp.isna().any().any())

# Count NaNs per column
print("\nNaN count per column in merged_output:")
print(generation_and_capacity_single_gsp.isna().sum())

# Count of rows with any NaN
n_rows_with_nan = generation_and_capacity_single_gsp.isna().any(axis=1).sum()
print(f"\nNumber of rows with any NaN: {n_rows_with_nan} out of {len(generation_and_capacity_single_gsp)}")

# Show all rows that contain any NaN (up to 20 for a quick check)
nan_rows = generation_and_capacity_single_gsp[generation_and_capacity_single_gsp.isna().any(axis=1)]



Any NaNs anywhere in merged_output: False

NaN count per column in merged_output:
datetime_gmt     0
gsp_id           0
generation_mw    0
GSPs             0
gsp_lat          0
gsp_lon          0
region_name      0
pes_id           0
GSP_region       0
month            0
year             0
day              0
hour             0
capacity_mwp     0
dtype: int64

Number of rows with any NaN: 0 out of 17520


#### Now let's check it with the historic weahter API df

In [254]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

def fetch_weather_for_generation_df(gen_df,
                                    cache_path=".cache",
                                    cache_expire=3600,
                                    retries=5,
                                    backoff_factor=0.2):
    """
    Extract lat/lon and date range from generation_and_capacity_single_gsp-like
    DataFrame (expects 'gsp_lat', 'gsp_lon', 'datetime_gmt'), call Open-Meteo via
    openmeteo_requests and return hourly weather DataFrame indexed by UTC datetime
    with added columns: year, month, day, hour.
    """
    # Extract lat/lon from first non-null row
    row = gen_df[['gsp_lat', 'gsp_lon']].dropna().iloc[0]
    latitude, longitude = float(row['gsp_lat']), float(row['gsp_lon'])

    # Derive start/end dates (YYYY-MM-DD)
    start_date = gen_df['datetime_gmt'].min().normalize().strftime('%Y-%m-%d')
    end_date = gen_df['datetime_gmt'].max().normalize().strftime('%Y-%m-%d')

    # Setup client with cache + retry
    cache_session = requests_cache.CachedSession(cache_path, expire_after=cache_expire)
    retry_session = retry(cache_session, retries=retries, backoff_factor=backoff_factor)
    client = openmeteo_requests.Client(session=retry_session)
    if not hasattr(client, "_close_session"):
        client._close_session = False  # avoid destructor AttributeError

    hourly_vars = [
        "shortwave_radiation", "direct_radiation", "diffuse_radiation",
        "direct_normal_irradiance", "global_tilted_irradiance", "terrestrial_radiation",
        "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant",
        "direct_normal_irradiance_instant", "global_tilted_irradiance_instant",
        "terrestrial_radiation_instant",
    ]

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": hourly_vars,
    }

    responses = client.weather_api(url, params=params)
    if not responses:
        return pd.DataFrame()

    response = responses[0]
    hourly = response.Hourly()
    hourly_arrays = [hourly.Variables(i).ValuesAsNumpy() for i in range(len(hourly_vars))]

    start_ts = pd.to_datetime(hourly.Time(), unit="s", utc=True)
    end_ts = pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True)
    interval_seconds = int(hourly.Interval())
    dates = pd.date_range(
        start=start_ts,
        end=end_ts,
        freq=pd.Timedelta(seconds=interval_seconds),
        inclusive="left"
    )

    data = {"date": dates}
    for name, arr in zip(hourly_vars, hourly_arrays):
        data[name] = arr[: len(dates)]

    df = pd.DataFrame(data).set_index("date")

    # Add year, month, day, hour columns from the index
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hour'] = df.index.hour

    return df

# Usage:
weather_df = fetch_weather_for_generation_df(generation_and_capacity_single_gsp)
weather_df.head()


,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant,year,month,day,hour
date,,,,,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,0
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,1
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,2
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,3
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,4


In [258]:
def merge_generation_with_weather(generation_df, weather_df):
    """
    Merge generation_and_capacity_single_gsp dataframe with weather_df.
    Only brings generation_mw and capacity_mwp from generation_df.
    - generation_df: must contain ['year','month','day','hour','generation_mw','capacity_mwp']
      (two half-hour rows per hour; this function averages per hour).
    - weather_df: indexed by UTC datetime and must contain columns ['year','month','day','hour']
    Returns: DataFrame with weather_df index and merged averaged columns:
             generation_mw, capacity_mwp (aligned to weather hourly rows).
    """
    # Select only required columns
    cols = ['year', 'month', 'day', 'hour', 'generation_mw', 'capacity_mwp']
    gen = generation_and_capacity_single_gsp[cols].copy()
    grouped = gen.groupby(['year', 'month', 'day', 'hour'], as_index=False)[['generation_mw', 'capacity_mwp']].mean() # Aggregate to hourly by mean
    weather = weather_df.copy().reset_index()  # bring datetime index to column named 'date'
    merged = weather.merge(grouped, on=['year', 'month', 'day', 'hour'], how='left') # Merge with weather_df on year,month,day,hour
    
    # Failsafe itnterpolate remaining NaN values in generation_mw and capacity_mwp
    if 'generation_mw' in merged.columns:
        merged['generation_mw'] = merged['generation_mw'].interpolate(limit_direction='both')

    if 'capacity_mwp' in merged.columns:
        merged['capacity_mwp'] = merged['capacity_mwp'].interpolate(limit_direction='both')
   
    # restore datetime index
    merged = merged.set_index('date')

    return merged
gen_weather_merged_df = merge_generation_with_weather(generation_and_capacity_single_gsp, weather_df)
gen_weather_merged_df

,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant,year,month,day,hour,generation_mw,capacity_mwp
date,,,,,,,,,,,,,,,,,,
2025-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,0,0.0,301.845517
2025-01-01 01:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,1,0.0,301.845517
2025-01-01 02:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,2,0.0,301.845517
2025-01-01 03:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,3,0.0,301.845517
2025-01-01 04:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,1,1,4,0.0,301.845517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-31 19:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,19,0.0,408.331217
2025-12-31 20:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,20,0.0,408.331217
2025-12-31 21:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,12,31,21,0.0,408.331217


In [259]:
# Check for NaNs in gen_weather_merged_df
print("Any NaNs anywhere in gen_weather_merged_df:", gen_weather_merged_df.isna().any().any())

# Count NaNs per column
print("\nNaN count per column in gen_weather_merged_df:")
print(gen_weather_merged_df.isna().sum())

# Count of rows with any NaN
n_rows_with_nan = gen_weather_merged_df.isna().any(axis=1).sum()
print(f"\nNumber of rows with any NaN: {n_rows_with_nan} out of {len(gen_weather_merged_df)}")

# Show all rows that contain any NaN (up to 20 for a quick check)
nan_rows = gen_weather_merged_df[gen_weather_merged_df.isna().any(axis=1)]
print("\nRows with NaNs (showing up to 20):")
print(nan_rows.head(20))


Any NaNs anywhere in gen_weather_merged_df: False

NaN count per column in gen_weather_merged_df:
shortwave_radiation                 0
direct_radiation                    0
diffuse_radiation                   0
direct_normal_irradiance            0
global_tilted_irradiance            0
terrestrial_radiation               0
shortwave_radiation_instant         0
direct_radiation_instant            0
diffuse_radiation_instant           0
direct_normal_irradiance_instant    0
global_tilted_irradiance_instant    0
terrestrial_radiation_instant       0
year                                0
month                               0
day                                 0
hour                                0
generation_mw                       0
capacity_mwp                        0
dtype: int64

Number of rows with any NaN: 0 out of 8760

Rows with NaNs (showing up to 20):
Empty DataFrame
Columns: [shortwave_radiation, direct_radiation, diffuse_radiation, direct_normal_irradiance, global_tilted_i